In [2]:
#!python3
'''
Maj Smith addition.  This is a greedy algo that just meets the requirements without any special consideration other than the min constraints.
Goal:  Pass the checker function.
'''
from DataLoader import *
from CheckSchedule import checkSchedule
from satUtils import satisfiable, satisfiableMember
import copy
from pprint import pprint
import random
import sys
sys.setrecursionlimit(10000)

def times_overlap(x1, x2, y1, y2) -> bool:
    return max(x1, y1) <= min(x2, y2)

def already_assigned_to_a_flight(member: int, event_dates: tuple) -> bool:
    try:
        assigned = schedule["Staff"][member]
        proposed_event_start = event_dates[0]
        proposed_event_end = event_dates[1]
    except:
        return False
    for flight in assigned:
        booked_start = events[flight]['StartDay']
        booked_end = events[flight]['EndDay']
        if times_overlap(proposed_event_start, proposed_event_end, booked_start, booked_end):
            return True
    return False

# assumption: unavailability begins at start day of event
def staff_on_leave(event: dict,
                       member: int,
                       member_details: dict) -> bool:
    # check if on leave
    if not member_details['Leave']:
        return False
    else:
        for dates in member_details['Leave']:
            leave_start = dates[0]
            leave_end = dates[-1]
            # if no overlap:
            if times_overlap(leave_start, leave_end, event['StartDay'], event['EndDay']):
                return True
    return False

def datenum_to_month(datenum_start: int) -> str:
    monthNames = ["Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    july = (60, 90)
    august = (91, 121)
    september = (122, 151)
    october = (152, 182)
    november = (183, 212)
    december = (213, 243)
    months = [july, august, september, october, november, december]
    monthDateNames = list(zip(months, monthNames))
    for days, month in monthDateNames:
        # in date range?
        if datenum_start >= days[0] and datenum_start <= days[-1]:
            return month
    print(datenum_start)
    print('Could not convert to month')
    
def schedule_wiggler(event_details: dict, n=3) -> dict:
    '''
    returns a dictionary that represents a single schedule, offset from its original by +/- n days
    '''
    new_start = float(random.choice(list(range(int(event_details['StartDay']-n), int(event_details['StartDay']+n+1)))))
    if new_start < 60:
        new_start = 60
    if new_start > 243:
        new_start = 243
    return {'Category': event_details['Category'],
            'CrewMax':  event_details['CrewMax'],
            'CrewMin':  event_details['CrewMin'],
            'CrewRequirements': event_details['CrewRequirements'],
            'CrewType': event_details['CrewType'],
            'EndDay': (new_start - event_details['StartDay']) 
                                    + event_details['EndDay'],
            'StartDay': new_start,
            'Type': event_details['Type']}

def check_aircrew_credentials(event: dict, available_staff: dict) -> list:
    crew_min = event['CrewMin']
    crew_max = event['CrewMax']
    crew_met = 0
    chosen_crew = []
    for req in event['CrewRequirements']:
        if req == '':
            # TODO: future change here! Optimize!
            while True:
                print('loop1')
                if crew_met >= crew_min:
                    break
                lucky_dude = random.choice(list(available_staff.items()))
                lucky_dude_id = lucky_dude[0]
                lucky_dude = {lucky_dude_id: lucky_dude[1]}
                # I think this exception has to do with new pilots showing up, and not having a record for prior months.
                month = datenum_to_month(event['StartDay'])
                try:
                    lucky_dude_qual = lucky_dude[lucky_dude_id]['Quals'][month]
                except KeyError:
                    lucky_dude_qual = lucky_dude[lucky_dude_id]['Quals']['Aug']
                if lucky_dude_qual == None:
                    pass
                # TODO: make sure these are the least qualified people:
                if lucky_dude not in chosen_crew and lucky_dude_qual in ['', None, 'none', 'FPNC', "FPCC","FPKC"]: # ,"FPQC","FPLC"
                    chosen_crew.append(lucky_dude)
                    crew_met += 1
                    break
        else:
            # TODO: future change here!  Optimize!
            counter = 0
            big_counter = 0
            while True:
                #print('loop 3')
                if crew_met >= crew_min:
                    break
                lucky_dude = random.choice(list(available_staff.items()))
                lucky_dude_id = lucky_dude[0]
                lucky_dude = {lucky_dude_id: lucky_dude[1]}
                month = datenum_to_month(event['StartDay'])
                # I think this exception has to do with new pilots showing up, and not having a record for prior months.
                try:
                    lucky_dude_qual = lucky_dude[lucky_dude_id]['Quals'][month]
                except KeyError:
                    lucky_dude_qual = lucky_dude[lucky_dude_id]['Quals']['Aug']
                if lucky_dude_qual == None:
                    continue
                if lucky_dude not in chosen_crew:
                    if satisfiableMember(req, lucky_dude_qual):
                        chosen_crew.append(lucky_dude)
                        crew_met += 1
                counter += 1
                big_counter += 1
                if counter > 40:
                    chosen_crew = []
                    counter = 0
                    crew_met = 0
                if big_counter > 1000:
                    return []
    assert(crew_met >= crew_min and crew_met <= crew_max)
    return chosen_crew

schedule = {
    "Events": {},
    "Staff": {},
}
events = loadEventData()
staff = loadPilotData()

def main():
    original_events = copy.deepcopy(events) # for record, if needed
    try:
        while len(schedule["Events"].keys()) < len(events):
            iter_events = copy.deepcopy(events)
            for event_name, event_details in iter_events.items():
                # check to see if we'be done this one before
                try:
                    test_exists = schedule['Events'][event_name]
                    if test_exists:
                        continue
                except:
                    pass
                staff_not_on_leave = {member: details for member, details in staff.items()
                    if not staff_on_leave(event_details, member, details)}
                staff_available = {member: details for member, details in staff_not_on_leave.items()
                    if not already_assigned_to_a_flight(member, (event_details['StartDay'], event_details['EndDay']))}
                # find crew with the credentials needed and meet min crew needs
                chosen_crew = check_aircrew_credentials(event_details, staff_available)
                if not chosen_crew:
                    events[event_name] = schedule_wiggler(event_details)
                    continue
                # schedule construction
                schedule['Events'][event_name] = event_details['StartDay']
                for member in chosen_crew:
                    for name in member.keys():
                        try:
                            schedule['Staff'][name].append(event_name)
                        except KeyError:
                            schedule['Staff'][name] = [event_name]
                print(len(schedule['Events']))
    except KeyboardInterrupt:
        pprint(schedule)
        checkSchedule(schedule, verbose=True)
    pprint(schedule)
    checkSchedule(schedule, verbose=True)
    
main()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
loop1
27

KeyboardInterrupt: 